## PyCrop##
A pure Python implementation of a simple empirical model of crop maturation, built from the FORTRAN DSSAT simple crop-soil water model exaple from
http://dssat.net/about/modular-approach-example

    Original FORTRAN code: RICARDO BRAGA and JADIR ROSA (for course AGE 5646-Agricultural and Biological Systems Simulation), date: 03/31/1997 Modified 07/1999 

Re-written in Python 3, by Benjamin A. Laken (benlaken@gmail.com) http://www.blaken.com

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from collections import OrderedDict

from PyCrop import *

#import matplotlib.pyplot as plt
#%matplotlib inline 
from bokeh.plotting import *
from bokeh.models import HoverTool
output_notebook()

Loading BokehJS ...

## Run a model ##

In [2]:
plant_doy=200                           # Planting doy
croptest = PyCrop(plant_doy)            # Create PyCrop class with the day of planting info
sw_out, p_out = croptest.simulate()     # Run a simulation using the weather data in a local folder

/Users/Ben/anaconda/lib/python3.5/site-packages/numpy/core/_methods.py:26: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return umr_maximum(a, axis, None, out, keepdims)


The crop matured on day 334
Simulation finished


In [3]:
# Get a dictionary of all the variables in the code
varlist = CustomFunctions.get_variable_defs()

In [4]:
# E.g. of how to use the dictionary to find a specific variable
lookup = 'Wr'
print('{0}: {1}'.format(lookup, varlist[lookup]))

Wr: root dry matter weight (g m-2)


## Explore Input data ##

In [5]:
TOOLS="pan,wheel_zoom,box_zoom,reset,crosshair,hover"
order = np.sort(croptest.weather.index)

# create a new plot
s1 = figure(width=600, plot_height=250, title='Weather data',x_axis_type="datetime",tools=TOOLS)
s1.circle(order, croptest.weather.tmax[order], size=3, color="red", alpha=0.5)
s1.circle(order, croptest.weather.tmin[order], size=3, color="blue", alpha=0.5)

# NEW: create a new plot and share both ranges
s2 = figure(width=600, height=250, x_range=s1.x_range, title=None,x_axis_type="datetime",tools=TOOLS)
s2.circle(order, croptest.weather.rain[order], size=3, color="green", alpha=0.5)

# NEW: create a new plot and share only one range
s3 = figure(width=600, height=250, x_range=s1.x_range, title=None,x_axis_type="datetime",tools=TOOLS)
s3.circle(order, croptest.weather.srad[order], size=3, color="red", alpha=0.5)
s3.circle(order, croptest.weather.par[order], size=3, color="orange", alpha=0.5)

s1.grid.grid_line_alpha=0
s1.yaxis.axis_label = 'T [°C]'
s1.ygrid.band_fill_color="grey"
s1.ygrid.band_fill_alpha = 0.1

s2.grid.grid_line_alpha=0
s2.yaxis.axis_label = 'rainfall [mm]'
s2.ygrid.band_fill_color="grey"
s2.ygrid.band_fill_alpha = 0.1

s3.grid.grid_line_alpha=0
s3.yaxis.axis_label = "Rad. [MJ/m²/dy]"
s3.ygrid.band_fill_color="grey"
s3.ygrid.band_fill_alpha = 0.1

# NEW: put the subplots in a gridplot
p = gridplot([[s1] ,[s2], [s3]])

# show the results
show(p)

## Explore Outputs ##

In [6]:
# create a new plot
o1 = figure(width=600, plot_height=250, title='PyCrop output',x_axis_type="datetime",tools=TOOLS)
o1.circle(sw_out.index, sw_out.SWC, size=3, color="red", alpha=0.5)
#s1.circle(order, croptest.weather.tmin[order], size=3, color="blue", alpha=0.5)

# NEW: create a new plot and share both ranges
o2 = figure(width=600, height=250, x_range=o1.x_range, title=None,x_axis_type="datetime",tools=TOOLS)
o2.circle(p_out.index, p_out.N, size=3, color="green", alpha=0.5)

# NEW: create a new plot and share only one range
o3 = figure(width=600, height=250, x_range=o1.x_range, title=None,x_axis_type="datetime",tools=TOOLS)
o3.circle(p_out.index, p_out.LAI, size=3, color="red", alpha=0.5)
#s3.circle(order, croptest.weather.par[order], size=3, color="orange", alpha=0.5)

o4 = figure(width=600, height=250, x_range=o1.x_range, title=None,x_axis_type="datetime",tools=TOOLS)
o4.circle(p_out.index, p_out.W, size=3, color="red", alpha=0.5,legend="Total")
o4.circle(p_out.index, p_out.Wc, size=3, color="blue", alpha=0.5,legend="Canopy")
o4.circle(p_out.index, p_out.Wf, size=3, color="green", alpha=0.5,legend="Fruit")
o4.circle(p_out.index, p_out.Wr, size=3, color="orange", alpha=0.5,legend="Roots")

o1.grid.grid_line_alpha=0
o1.yaxis.axis_label = 'Soil water content'
o1.ygrid.band_fill_color="grey"
o1.ygrid.band_fill_alpha = 0.1

o2.grid.grid_line_alpha=0
o2.yaxis.axis_label = 'Leaf number'
o2.ygrid.band_fill_color="grey"
o2.ygrid.band_fill_alpha = 0.1

o3.grid.grid_line_alpha=0
o3.yaxis.axis_label = "LAI"
o3.ygrid.band_fill_color="grey"
o3.ygrid.band_fill_alpha = 0.1

o4.grid.grid_line_alpha=0
o4.yaxis.axis_label = "Weight [g/m²]"
o4.ygrid.band_fill_color="grey"
o4.ygrid.band_fill_alpha = 0.1

# NEW: put the subplots in a gridplot
p2 = gridplot([[o1] ,[o2], [o3],[o4]])

#hover = p.select(dict(type=HoverTool))


# show the results
show(p2)